# HouseBrain: Production Fine-Tuning on Google Colab

This notebook provides the complete, end-to-end workflow for fine-tuning a production-grade Large Language Model for architectural design. It uses our full **20-example "Gold Standard" dataset** to teach a powerful base model (`meta-llama/Llama-2-7b-chat-hf`) the specific schema and nuances of Indian residential architecture.

**GPU Requirement:** This notebook is designed for a T4 GPU, which is available in the free tier of Google Colab. For faster results, you can use an A100 on Colab Pro+.


## Step 1: Environment Setup

This step clones the project repository from GitHub and installs all the necessary Python packages for fine-tuning, including `transformers`, `peft`, `trl`, and `bitsandbytes` for memory-efficient 4-bit training.


In [ ]:
!git clone https://github.com/your-username/housebrain_v1_1.git
%cd housebrain_v1_1

!pip install -q -U transformers datasets accelerate peft trl bitsandbytes


## Step 3: (Optional) Generate New High-Quality Drafts

This section is for creating new **Gold** or **Platinum** standard examples. It will set up an Ollama server within the Colab environment, download a powerful base model (`deepseek-r1:8b`), and use it to generate raw drafts based on expert prompts.

**Workflow:**
1.  Run the cells below to generate the raw `.json` draft files.
2.  Download the generated files from the Colab file browser (under `data/training/gold_standard/` or `data/training/platinum_standard/`).
3.  Use the AI assistant's "Analyze and Repair" process to perfect the drafts locally.
4.  Upload the final, corrected `.json` files back to the appropriate directory before proceeding to the next step.


In [ ]:
# Install Ollama
!if ! command -v ollama &> /dev/null; then curl -fsSL https://ollama.com/install.sh | sh; fi

# Start Ollama in the background
import os
import asyncio

async def run_process(cmd):
  print('>>>', cmd)
  proc = await asyncio.create_subprocess_shell(
      cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE)
  
  # Log output in real-time
  while True:
      line = await proc.stdout.readline()
      if not line:
          break
      print(line.decode().strip())
      
  await proc.wait()

# Start ollama serve in the background
asyncio.create_task(run_process('ollama serve > ollama.log 2>&1'))

# Wait for Ollama to be ready
import time
import requests

print("⏳ Waiting for Ollama server to start...")
time.sleep(5) # Initial wait
for i in range(60): # Wait up to 60 seconds
    try:
        requests.get("http://localhost:11434")
        print("✅ Ollama server is running!")
        break
    except requests.exceptions.ConnectionError:
        time.sleep(1)
else:
    print("❌ Ollama server failed to start. Check ollama.log for errors.")
    !cat ollama.log

# Download the model
!ollama pull deepseek-r1:8b


In [ ]:
# Create the Platinum Standard directory if it doesn't exist
!mkdir -p data/training/platinum_standard

# --- GENERATE GOLD STANDARD DRAFT #21 ---
GOLD_PROMPT = "Design a luxurious 4BHK G+1 duplex for a 40x60 feet west-facing plot in a gated community in Bangalore. The design must be Vastu-compliant and include a home office on the ground floor, a private family lounge on the first floor, and balconies for every bedroom. The client desires a contemporary architectural style with large windows for ample natural light."
!python scripts/generate_draft_from_prompt.py --scenario "{GOLD_PROMPT}" --output-file "data/training/gold_standard/gold_standard_21_draft.json"

print("\\n" + "="*50 + "\\n")

# --- GENERATE PLATINUM STANDARD DRAFT #01 ---
PLATINUM_PROMPT = "Design a one-of-a-kind, 'biophilic' 3BHK luxury retreat on a 50x80 feet plot overlooking the backwaters of Kerala. The design must seamlessly integrate indoor and outdoor spaces, featuring a central open-to-sky courtyard with a water body, extensive use of natural materials like laterite stone and teak wood, and a cantilevered infinity pool on the first floor. Prioritize sustainability with rainwater harvesting and solar panel provisions. The architectural style should be a modern interpretation of traditional Kerala design."
!python scripts/generate_draft_from_prompt.py --scenario "{PLATINUM_PROMPT}" --output-file "data/training/platinum_standard/platinum_standard_01_draft.json"


## Step 4: Prepare the Gold Standard Data

This step runs our preparation script. It will process the 20 raw Gold Standard JSON files (plus any new ones you've generated and perfected) and create a new `gold_standard_finetune_ready` directory containing the data in the simple `{"prompt": "...", "output": "..."}` format required by the training script.


In [ ]:
!python scripts/prepare_gold_standard_data.py
!echo "\n✅ Data preparation complete. Verifying the new directory:"
!ls -l data/training/gold_standard_finetune_ready | wc -l


## Step 5: Run the Fine-Tuning Script

This is the core of the process. We execute the `run_finetuning.py` script, which will:

1.  **Load** our 20 prepared Gold Standard examples.
2.  **Download** the base `Llama-2-7b-chat-hf` model from Hugging Face.
3.  **Configure** 4-bit quantization and LoRA for efficient training.
4.  **Fine-tune** the model on our data.
5.  **Save** the final, specialized `housebrain-llama2-7b-v0.1` model to the `models/` directory.

We will use a high number of epochs (e.g., 200) because our dataset is very high-quality but small. This is necessary to ensure the model learns the schema thoroughly.


In [ ]:
!python scripts/run_finetuning.py \
    --dataset-path "data/training/gold_standard_finetune_ready" \
    --base-model "meta-llama/Llama-2-7b-chat-hf" \
    --output-path "models/housebrain-llama2-7b-v0.1" \
    --epochs 200 \
    --batch-size 2 \
    --learning-rate 2e-5


## Step 6: Next Steps - Using Your Fine-Tuned Model

Once training is complete, the new model is saved in the `models/housebrain-llama2-7b-v0.1` directory. 

You can now use this specialized model in your `generate_validated_silver_data.py` script (by changing the model ID) to generate a large, high-quality dataset of thousands of examples. This is the path to a truly production-ready system.


## Step 7 (Optional): A/B Test with a State-of-the-Art Competitor (Qwen2)

To ensure we are using the absolute best base model, you can run this second experiment to fine-tune the new, high-performing `Qwen2-7B-Instruct` model on our same Gold Standard dataset.

Once this is trained, you will have two models: `housebrain-llama2-7b-v0.1` and `housebrain-qwen2-7b-v0.1`. You can then evaluate them head-to-head on a new set of prompts to see which one produces superior architectural designs. This data-driven approach guarantees we select the best possible foundation for our production system.


In [ ]:
# !python scripts/run_finetuning.py \
#     --dataset-path "data/training/gold_standard_finetune_ready" \
#     --base-model "Qwen/Qwen2-7B-Instruct" \
#     --output-path "models/housebrain-qwen2-7b-v0.1" \
#     --epochs 200 \
#     --batch-size 2 \
#     --learning-rate 2e-5
